# Auto Agent + RAG Documents Demo

This notebook demonstrates how to **upload local documents to RAG** and use the **Auto Agent** to query RAG document collections.

Key concepts:
- Upload your own local documents (PDF, TXT, MD, DOCX, CSV, etc.) to RAG collections
- Browse and inspect RAG collections and their documents
- Manage collections (create, delete, remove specific documents)
- Use the `auto` agent type which intelligently routes queries to the appropriate agent
- When the query involves document retrieval, the auto agent selects the **ReAct agent**, which calls the **RAG tool**

### How the Auto Agent Routes to RAG

```
User Query
  -> Auto Agent (LLM decides: chat / react / plan_execute)
    -> ReAct Agent (if tools are needed)
      -> RAG Tool (if query involves internal documents)
        -> Semantic Search over existing collection
          -> LLM synthesizes answer from retrieved chunks
```

### Prerequisites

1. Both servers must be running (`python tools_server.py` then `python server.py`)
2. At least one RAG collection must already exist with uploaded documents
3. `RAG_DEFAULT_COLLECTION` in `config.py` must match the target collection name

## Setup and Configuration

In [ ]:
import httpx
import json
from IPython.display import display, Markdown


class LLMApiClient:
    """Unified client for the LLM API server."""

    def __init__(self, base_url: str, timeout: float = 6000.0):
        self.base_url = base_url.rstrip("/")
        self.token = None
        self.timeout = httpx.Timeout(50.0, read=timeout, write=timeout, pool=timeout)

    def _headers(self):
        return {"Authorization": f"Bearer {self.token}"} if self.token else {}

    # ---- Auth ----

    def login(self, username: str, password: str):
        r = httpx.post(
            f"{self.base_url}/api/auth/login",
            json={"username": username, "password": password},
            timeout=10.0,
        )
        r.raise_for_status()
        self.token = r.json()["access_token"]
        return r.json()

    def list_models(self):
        r = httpx.get(f"{self.base_url}/v1/models", headers=self._headers(), timeout=10.0)
        r.raise_for_status()
        return r.json()

    # ---- Chat (auto agent) ----

    def chat_new(self, model: str, user_message: str, agent_type: str = "auto"):
        """Start a new chat session. Returns (response_text, session_id)."""
        messages = [{"role": "user", "content": user_message}]
        data = {
            "model": model,
            "messages": json.dumps(messages),
            "agent_type": agent_type,
        }
        r = httpx.post(
            f"{self.base_url}/v1/chat/completions",
            data=data,
            headers=self._headers(),
            timeout=self.timeout,
        )
        r.raise_for_status()
        result = r.json()
        return result["choices"][0]["message"]["content"], result["x_session_id"]

    def chat_continue(self, model: str, session_id: str, user_message: str, agent_type: str = "auto"):
        """Continue an existing session. Returns (response_text, session_id)."""
        messages = [{"role": "user", "content": user_message}]
        data = {
            "model": model,
            "messages": json.dumps(messages),
            "session_id": session_id,
            "agent_type": agent_type,
        }
        r = httpx.post(
            f"{self.base_url}/v1/chat/completions",
            data=data,
            headers=self._headers(),
            timeout=self.timeout,
        )
        r.raise_for_status()
        result = r.json()
        return result["choices"][0]["message"]["content"], result["x_session_id"]

    # ---- RAG management (direct tools-server calls) ----

    def rag_list_collections(self, tools_base: str):
        """List all RAG collections for the authenticated user."""
        r = httpx.get(
            f"{tools_base}/api/tools/rag/collections",
            headers=self._headers(),
            timeout=10.0,
        )
        r.raise_for_status()
        return r.json()

    def rag_list_documents(self, tools_base: str, collection_name: str):
        """List documents in a RAG collection."""
        r = httpx.get(
            f"{tools_base}/api/tools/rag/collections/{collection_name}/documents",
            headers=self._headers(),
            timeout=10.0,
        )
        r.raise_for_status()
        return r.json()

    def rag_query_direct(self, tools_base: str, query: str, collection_name: str, max_results: int = 5):
        """Query RAG directly via the tools server (bypasses the agent)."""
        r = httpx.post(
            f"{tools_base}/api/tools/rag/query",
            headers=self._headers(),
            json={
                "query": query,
                "collection_name": collection_name,
                "max_results": max_results,
            },
            timeout=self.timeout,
        )
        r.raise_for_status()
        return r.json()

    def rag_upload_document(self, tools_base: str, collection_name: str, file_path: str):
        """Upload a local document to a RAG collection."""
        from pathlib import Path
        
        file_path = Path(file_path)
        if not file_path.exists():
            raise FileNotFoundError(f"File not found: {file_path}")
        
        with open(file_path, "rb") as f:
            files = {"file": (file_path.name, f, "application/octet-stream")}
            data = {"collection_name": collection_name}
            
            r = httpx.post(
                f"{tools_base}/api/tools/rag/upload",
                headers=self._headers(),
                files=files,
                data=data,
                timeout=self.timeout,
            )
        
        r.raise_for_status()
        return r.json()

    def rag_create_collection(self, tools_base: str, collection_name: str):
        """Create a new RAG collection."""
        r = httpx.post(
            f"{tools_base}/api/tools/rag/collections",
            headers=self._headers(),
            json={"collection_name": collection_name},
            timeout=10.0,
        )
        r.raise_for_status()
        return r.json()

    def rag_delete_collection(self, tools_base: str, collection_name: str):
        """Delete a RAG collection."""
        r = httpx.delete(
            f"{tools_base}/api/tools/rag/collections/{collection_name}",
            headers=self._headers(),
            timeout=10.0,
        )
        r.raise_for_status()
        return r.json()

    def rag_delete_document(self, tools_base: str, collection_name: str, document_id: str):
        """Delete a specific document from a collection."""
        r = httpx.delete(
            f"{tools_base}/api/tools/rag/collections/{collection_name}/documents/{document_id}",
            headers=self._headers(),
            timeout=10.0,
        )
        r.raise_for_status()
        return r.json()


# -------------------------------------------------------------------
# Configuration  (adjust to your environment)
# -------------------------------------------------------------------
API_BASE_URL   = "http://localhost:10007"   # Main API server
TOOLS_BASE_URL = "http://localhost:10006"   # Tools API server
USERNAME = "admin"
PASSWORD = "administrator"

client = LLMApiClient(API_BASE_URL, timeout=600.0)
print("\u2713 Client initialized")
print(f"  Main server : {API_BASE_URL}")
print(f"  Tools server: {TOOLS_BASE_URL}")

## Step 1: Authenticate and Discover Model

In [ ]:
client.login(USERNAME, PASSWORD)
models = client.list_models()
MODEL = models["data"][0]["id"]

print(f"\u2713 Logged in as: {USERNAME}")
print(f"\u2713 Using model : {MODEL}")

## Step 2: Upload Local Documents to RAG (Optional)

If you want to add your own documents to RAG, use this section. Skip to Step 3 if you already have documents uploaded.

### Supported File Formats

- **Text**: `.txt`, `.md`
- **Documents**: `.pdf`, `.docx`
- **Data**: `.json`, `.csv`, `.xlsx`, `.xls`

In [ ]:
# Create a new RAG collection and upload documents
from pathlib import Path

# Step 1: Create collection
collection_name = "default"  # Change this to your desired collection name

try:
    print(f"Creating collection '{collection_name}'...")
    result = client.rag_create_collection(TOOLS_BASE_URL, collection_name)
    
    if result.get("success"):
        print(f"✓ Collection created successfully!")
        print(f"  Collection name: {collection_name}\n")
    else:
        print(f"✗ Failed to create collection: {result.get('error')}\n")
except Exception as e:
    print(f"✗ Error creating collection: {e}\n")

# Step 2: Upload your PDF files
# Replace with your actual PDF file paths
custom_pdf_files = [
    "./USB 3.2 Revision 1.1.pdf",
    "./usb_20.pdf"
]

print(f"Uploading {len(custom_pdf_files)} documents...\n")

for pdf_file in custom_pdf_files:
    # Check if file exists
    if not Path(pdf_file).exists():
        print(f"⚠️  File not found: {pdf_file}\n")
        continue
    
    print(f"📤 Uploading: {pdf_file}")
    
    try:
        result = client.rag_upload_document(TOOLS_BASE_URL, collection_name, pdf_file)
        
        if result.get('success'):
            print(f"  ✓ Success! Chunks created: {result.get('chunks_created')}")
            print(f"  Total chunks in collection: {result.get('total_chunks')}\n")
        else:
            print(f"  ✗ Failed: {result.get('error')}\n")
    except Exception as e:
        print(f"  ✗ Error: {str(e)}\n")

## Step 3: Browse Existing RAG Collections

Let's see which collections and documents are available after any uploads.

In [ ]:
collections_result = client.rag_list_collections(TOOLS_BASE_URL)

if collections_result.get("success"):
    collections = collections_result["collections"]
    print(f"Found {len(collections)} collection(s):\n")
    for coll in collections:
        print(f"  Collection : {coll['name']}")
        print(f"  Documents  : {coll['documents']}")
        print(f"  Chunks     : {coll['chunks']}")
        print(f"  Created    : {coll['created_at']}")
        print()
else:
    print("ERROR: Could not list collections.")
    print(collections_result)

## Step 4: Inspect Documents in a Collection

Pick the collection you want the auto agent to query.  
Set `COLLECTION_NAME` below to match one of the collections listed above.

In [ ]:
# === Set this to the collection you want to use ===
COLLECTION_NAME = "default"

docs_result = client.rag_list_documents(TOOLS_BASE_URL, COLLECTION_NAME)

if docs_result.get("success"):
    print(f"Collection   : {docs_result['collection_name']}")
    print(f"Total docs   : {docs_result['total_documents']}")
    print(f"Total chunks : {docs_result['total_chunks']}")
    print(f"\nDocuments:")
    for doc in docs_result["documents"]:
        print(f"  - {doc['name']}  ({doc['chunks']} chunks, uploaded {doc['uploaded_at']})")
else:
    print(f"ERROR: Could not list documents in '{COLLECTION_NAME}'.")
    print(docs_result)

## Step 5: Verify RAG_DEFAULT_COLLECTION

The **Auto Agent -> ReAct -> RAG tool** pipeline uses `RAG_DEFAULT_COLLECTION` from `config.py` to decide which collection to search.  
Make sure the value in `config.py` matches the collection you want to query **before starting the servers**.

```python
# In config.py:
RAG_DEFAULT_COLLECTION = "default"  # <-- must match your target collection
```

If you need to change it, edit `config.py` and restart both servers.

In [ ]:
# Quick sanity check: read the current config value
import importlib, sys

# Add project root so we can import config
sys.path.insert(0, "..")
import config as cfg
importlib.reload(cfg)

print(f"RAG_DEFAULT_COLLECTION = \"{cfg.RAG_DEFAULT_COLLECTION}\"")
print(f"Target collection      = \"{COLLECTION_NAME}\"")

if cfg.RAG_DEFAULT_COLLECTION == COLLECTION_NAME:
    print("\n\u2713 Config matches. The auto agent will query the correct collection.")
else:
    print(f"\n\u2717 MISMATCH! Edit config.py to set RAG_DEFAULT_COLLECTION = \"{COLLECTION_NAME}\"")
    print("  Then restart both servers.")

## Step 6: Query with the Auto Agent

Now send a question through the **auto agent**.  
The auto agent will:
1. Analyze the query and decide it needs document retrieval
2. Route to the **ReAct agent**
3. The ReAct agent calls the **RAG tool** to search the collection
4. Return a synthesized answer based on retrieved document chunks

> **Tip**: Phrasing the query to mention "documents" or "internal docs" helps the ReAct agent select the RAG tool over web search.

In [ ]:
# Initialize comparison utilities
from IPython.display import HTML
import time

def compare_responses(query, collection_name, model, session_id=None):
    """Execute query on both RAG and Auto Agent, display side-by-side comparison"""
    
    print(f"Query: {query}\n")
    print("=" * 140)
    
    # --- Direct RAG Query ---
    rag_start = time.time()
    direct_result = client.rag_query_direct(
        TOOLS_BASE_URL,
        query=query,
        collection_name=collection_name,
        max_results=5,
    )
    rag_time = time.time() - rag_start
    
    # --- Auto Agent Query ---
    agent_start = time.time()
    if session_id is None:
        agent_response, new_session_id = client.chat_new(
            model=model,
            user_message=query,
            agent_type="auto",
        )
    else:
        agent_response, new_session_id = client.chat_continue(
            model=model,
            session_id=session_id,
            user_message=query,
            agent_type="auto",
        )
    agent_time = time.time() - agent_start
    
    # --- Format RAG Result ---
    if direct_result.get("success"):
        rag_answer = direct_result['answer']
        data = direct_result.get("data", {})
        rag_metadata = f"""
        <div style='font-size: 0.9em; color: #666; margin-top: 10px;'>
        ✓ Results: {data.get('num_results', 0)} chunks retrieved<br>
        ✓ Exec time: {rag_time:.2f}s<br>
        <strong>Sources:</strong><br>
        """
        for i, doc in enumerate(data.get("documents", []), 1):
            rag_metadata += f"  [{i}] {doc['document']} chunk {doc['chunk_index']} (score {doc.get('score', 0):.3f})<br>"
        rag_metadata += "</div>"
    else:
        rag_answer = f"ERROR: {direct_result.get('error')}"
        rag_metadata = ""
    
    # --- Format Agent Result ---
    agent_metadata = f"""
    <div style='font-size: 0.9em; color: #666; margin-top: 10px;'>
    ✓ Exec time: {agent_time:.2f}s<br>
    ✓ Session ID: {new_session_id}
    </div>
    """
    
    # --- Display Side-by-Side ---
    html_output = f"""
    <div style='display: flex; gap: 20px; margin-top: 20px;'>
        <div style='flex: 1; border: 2px solid #4CAF50; border-radius: 8px; padding: 15px; background-color: #f9f9f9;'>
            <h3 style='color: #4CAF50; margin-top: 0;'>🔍 Direct RAG Query</h3>
            <div style='background: white; padding: 10px; border-radius: 5px; margin-bottom: 10px;'>
                {rag_answer.replace(chr(10), '<br>')}
            </div>
            {rag_metadata}
        </div>
        <div style='flex: 1; border: 2px solid #2196F3; border-radius: 8px; padding: 15px; background-color: #f9f9f9;'>
            <h3 style='color: #2196F3; margin-top: 0;'>🤖 Auto Agent Query</h3>
            <div style='background: white; padding: 10px; border-radius: 5px; margin-bottom: 10px;'>
                {agent_response.replace(chr(10), '<br>')}
            </div>
            {agent_metadata}
        </div>
    </div>
    """
    
    display(HTML(html_output))
    print("=" * 140 + "\n")
    
    return new_session_id

# Initialize session for first query
print("✓ Comparison utility loaded")

## Question 1: C-PHY Insertion Loss at 3.9Gsps

In [ ]:
query_1 = "C-PHY가 3.9Gsps로 동작할 때 Insertion Loss 스펙을 알려줘"
session_id = compare_responses(query_1, COLLECTION_NAME, MODEL)

## Question 2: MIPI C-PHY SKEW Error

In [ ]:
query_2 = "MIPI C-PHY에서 SKEW Error에 대해 알려줘"
session_id = compare_responses(query_2, COLLECTION_NAME, MODEL, session_id)

## Question 3: MIPI M-PHY v6.0 FOM and EOM

In [ ]:
query_3 = "MIPI M-PHY v6.0의 FOM과 EOM에 대해서 알려줘. 각각에 대해 비교해주고, Spec.에서는 Mandatory사항인지, Option사항인지 확인해줘"
session_id = compare_responses(query_3, COLLECTION_NAME, MODEL, session_id)

## Question 4: USB3.2 LTSSM and RX.Detect

In [ ]:
query_4 = "USB3.2의 LTSSM에 대해서 자세히 설명해주고, 특히 RX.Detect에 대해서 자세히 설명해줘"
session_id = compare_responses(query_4, COLLECTION_NAME, MODEL, session_id)

## Question 5: PCIe AER (Advanced Error Reporting)

In [ ]:
query_5 = "PCIe AER에 대해서 설명해주고, 대표적인 Error 들에 대해서 비교 설명해줘"
session_id = compare_responses(query_5, COLLECTION_NAME, MODEL, session_id)

## Question 6: USB3.2 Gen1/Gen2 Transmitter Eye Mask

In [ ]:
query_6 = "USB3.2의 Gen1(5GT/s), Gen2(10GT/s)의 Transmitter eye mask 규격에 대해서 측정 위치와 Eye Height spec.(Minimum, Maximum)에 대해서 비교 설명해줘"
session_id = compare_responses(query_6, COLLECTION_NAME, MODEL, session_id)

## Question 7: DisplayPort Link Training

In [ ]:
query_7 = "DP Source와 Sink간 Link training 절차에 대해서 설명해주고, Clock Recovery, Channel equalization, Symbol Lock을 수행하는 프로세스에 대해서 알려줘"
session_id = compare_responses(query_7, COLLECTION_NAME, MODEL, session_id)

## Question 8: USB2.0 High-Speed Disconnect Level (VHSDSC)

In [ ]:
query_8 = "USB2.0의 high-speed Disconnect level(VHSDSC)에 대해서 설명해주고, 최초 USB2.0 spec.대비 USB 2.0 DCR ECN에 의해 변경된 사항에 대해 알려줘"
session_id = compare_responses(query_8, COLLECTION_NAME, MODEL, session_id)

## Question 9: USB4.0, USB3.2, USB2.0 Comparison

In [ ]:
query_9 = "USB4.0, USB3.2, USB2.0 각각의 Spec.에 대해서 신호를 전송하는 전송방식과 최대 동작속도와 물리적인 배선의 차이점에 대해서 설명해줘"
session_id = compare_responses(query_9, COLLECTION_NAME, MODEL, session_id)

## Question 10: UFS, Unipro AFCx Frame Transmission

In [ ]:
query_10 = "UFS, Unipro의 AFCx Frame Transmission에 대한 설명을 해주고, FCx_PROTECTION_TIMER or TCx_REPLAY_TIMER에 대해서도 설명해줘"
session_id = compare_responses(query_10, COLLECTION_NAME, MODEL, session_id)

## Question 11: UFS, MIPI MPHY, Unipro Link Initialization

In [ ]:
query_11 = "UFS는 MIPI MPHY, Unipro로 이루어지는데 각각에 대해서 설명을 해주고, Link initialization process에 대해서 확인해줘"
session_id = compare_responses(query_11, COLLECTION_NAME, MODEL, session_id)

## Question 12: DisplayPort 1.4 Voltage Swing and Pre-emphasis

In [ ]:
query_12 = "DP1.4 spec.에서 정의하는 Voltage Swing and Pre-emphasis에 대해서 설명해주고, Voltage swing level과 TX Emphasis Level이 0/1/2/3으로 설정되는 이유에 대해서 알려줘"
session_id = compare_responses(query_12, COLLECTION_NAME, MODEL, session_id)

## Question 13: MIPI C-PHY Crosstalk Spec

In [ ]:
query_13 = "MIPI C-PHY crosstalk spec은?"
session_id = compare_responses(query_13, COLLECTION_NAME, MODEL, session_id)

## Question 14: Interface Impedance Guide

In [ ]:
query_14 = "인터페이스별 임피던스 가이드를 알려줘"
session_id = compare_responses(query_14, COLLECTION_NAME, MODEL, session_id)

## Question 15: MIPI D-PHY Initial vs Periodic Skew Calibration

In [ ]:
query_15 = "MIPI D-phy에서 Initial Skew cal과 Periodic Skew cal의 차이에 대해서 설명해줘"
session_id = compare_responses(query_15, COLLECTION_NAME, MODEL, session_id)

## Question 16: USB3.2 Gen2 Electrical Compliance Test

In [ ]:
query_16 = "USB3.2 Gen2 동작 시의 Electrical Compliance Test 항목에 대해서 알려줘"
session_id = compare_responses(query_16, COLLECTION_NAME, MODEL, session_id)

## Question 17: eUSB Repeater Mode eSE1 Signal

In [ ]:
query_17 = "eUSB의 Repeater mode에서 eSE1 신호가 발생하는 경우에 대해 설명해 주세요"
session_id = compare_responses(query_17, COLLECTION_NAME, MODEL, session_id)

## Question 18: USB vs eUSB FS Signaling Difference

In [ ]:
query_18 = "USB와 eUSB의 FS signaling 차이를 설명해 주세요"
session_id = compare_responses(query_18, COLLECTION_NAME, MODEL, session_id)

## Question 19: USB3.0 Voltage Level

In [ ]:
query_19 = "USB3.0 전압 레벨은 어떻게되지?"
session_id = compare_responses(query_19, COLLECTION_NAME, MODEL, session_id)

## Question 20: USB2.0 EQ Impact on Design Length

In [ ]:
query_20 = "USB2.0에서 EQ을 사용한 경우와 사용하지 않은 경우에 설계 길이의 차이는 얼마나 될까?"
session_id = compare_responses(query_20, COLLECTION_NAME, MODEL, session_id)

## Question 21: USB3.2 Gen2x2 8b/10b Encoding Maximum Speed

In [ ]:
query_21 = "USB3.2 Gen2x2 에서 8b/10b 인코딩 방식을 사용할때 최대 전송 속도는 얼마가 될까?"
session_id = compare_responses(query_21, COLLECTION_NAME, MODEL, session_id)

## Step 7: Follow-Up Query (Same Session)

Continue the conversation in the same session.  
The agent retains previous context so follow-up questions work naturally.

In [ ]:
followup = "Based on the documents, give me a more detailed summary of the key technical specifications."

print(f"Follow-up: {followup}")
print("Waiting for response...\n")

response2, _ = client.chat_continue(
    model=MODEL,
    session_id=session_id,
    user_message=followup,
    agent_type="auto",
)

display(Markdown(response2))

## Step 8: Compare — Direct RAG Query vs Auto Agent

To illustrate the difference, here we call the RAG tool **directly** (bypassing the agent) and compare it with the auto agent response for the same question.

In [ ]:
comparison_query = "C-PHY가 3.9Gsps로 동작할 때 Insertion Loss 스펙을 알려줘"

# --- Direct RAG query (tools server) ---
print("=" * 70)
print("[A] Direct RAG Query (via tools server)")
print("=" * 70)

direct_result = client.rag_query_direct(
    TOOLS_BASE_URL,
    query=comparison_query,
    collection_name=COLLECTION_NAME,
    max_results=5,
)

if direct_result.get("success"):
    display(Markdown(f"**Answer:**\n\n{direct_result['answer']}"))
    data = direct_result.get("data", {})
    print(f"\nResults: {data.get('num_results', 0)} chunks retrieved")
    print(f"Exec time: {direct_result.get('metadata', {}).get('execution_time', 0):.2f}s")
    for i, doc in enumerate(data.get("documents", []), 1):
        print(f"  [{i}] {doc['document']} chunk {doc['chunk_index']} (score {doc.get('score', 0):.3f})")
else:
    print(f"ERROR: {direct_result.get('error')}")

## Summary

This notebook demonstrated:

1. **Uploading local documents to RAG** — Add your own documents (PDF, TXT, MD, DOCX, CSV, etc.) to RAG collections
2. **Browsing existing RAG collections** — List collections and inspect their documents
3. **Config verification** — Ensure `RAG_DEFAULT_COLLECTION` points to the right collection
4. **Auto Agent routing** — The `auto` agent analyzes the query and routes to `react` when document retrieval is needed
5. **Multi-turn conversation** — Follow-up questions within the same session retain context
6. **Direct vs Agent comparison** — Direct RAG calls are stateless and explicit; auto agent adds reasoning and session memory
7. **Collection management** — Create/delete collections and remove specific documents

### Agent Routing Flow

```
auto agent
  ├── "chat"          → Simple Q&A (no tools)
  ├── "react"         → Single tool call (websearch / python_coder / rag)
  └── "plan_execute"  → Multi-step tasks with planning
```

### Tips for Reliable RAG Routing

- Include phrases like *"search internal documents"*, *"from the documents"*, or *"based on uploaded files"* in your query
- The ReAct system prompt instructs: *"Use rag only when explicitly asked to search internal documents"*
- If the auto agent routes to `chat` instead of `react`, rephrase to make the document-search intent explicit

### API Endpoints Used

| Endpoint | Purpose |
|----------|--------|
| `POST /api/auth/login` | Authentication |
| `GET /v1/models` | Discover available models |
| `POST /api/tools/rag/upload` | Upload document to RAG collection |
| `POST /api/tools/rag/collections` | Create new collection |
| `GET /api/tools/rag/collections` | List existing collections |
| `GET /api/tools/rag/collections/{name}/documents` | List documents in a collection |
| `DELETE /api/tools/rag/collections/{name}/documents/{id}` | Delete specific document |
| `DELETE /api/tools/rag/collections/{name}` | Delete entire collection |
| `POST /api/tools/rag/query` | Direct RAG query (tools server) |
| `POST /v1/chat/completions` | Chat via auto agent (main server) |

## Step 10: Collection Management (Optional)

You can also create new collections, delete collections, or delete specific documents.

In [ ]:
# Create a new collection
# Uncomment to create

# new_collection_name = "my_new_collection"
# try:
#     result = client.rag_create_collection(TOOLS_BASE_URL, new_collection_name)
#     if result.get("success"):
#         print(f"✓ Collection '{new_collection_name}' created successfully")
#     else:
#         print(f"✗ Failed: {result.get('error')}")
# except Exception as e:
#     print(f"✗ Error: {e}")

In [ ]:
# Delete a document from a collection
# First, list documents to get the document ID, then uncomment to delete

# collection_name = "default"
# document_id = "abc123..."  # Get this from list_documents result

# try:
#     result = client.rag_delete_document(TOOLS_BASE_URL, collection_name, document_id)
#     if result.get("success"):
#         print(f"✓ Document deleted successfully")
#         print(f"  Deleted: {result.get('deleted_document')}")
#         print(f"  Remaining documents: {result.get('remaining_documents')}")
#     else:
#         print(f"✗ Failed: {result.get('error')}")
# except Exception as e:
#     print(f"✗ Error: {e}")

In [ ]:
# Delete an entire collection (use with caution!)
# Uncomment to delete

# collection_to_delete = "test_collection"
# try:
#     result = client.rag_delete_collection(TOOLS_BASE_URL, collection_to_delete)
#     if result.get("success"):
#         print(f"✓ Collection '{collection_to_delete}' deleted successfully")
#     else:
#         print(f"✗ Failed: {result.get('error')}")
# except Exception as e:
#     print(f"✗ Error: {e}")